In [1]:
import torch
import gc
from transformers import AutoTokenizer, LlamaForCausalLM


gc.collect()
torch.cuda.empty_cache()
model_directory = "/home/ksaff/Desktop/ttyd/query_llama"
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = LlamaForCausalLM.from_pretrained(model_directory,
                                         load_in_8bit=True,
                                         device_map={'': 0})

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.05s/it]


In [2]:
ddl = """CREATE TABLE Customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    DOB DATE
);

CREATE TABLE Orders (
    orderID INTEGER PRIMARY KEY AUTOINCREMENT,
    customerID INTEGER,
    productID INTEGER,
    quantity INTEGER,
    orderDate TIMESTAMP
);

CREATE TABLE Products (
    productID INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    category TEXT,
    price DECIMAL(10, 2),
    inStock INTEGER
);

CREATE TABLE Employees (
    empID INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT,
    lastName TEXT,
    position TEXT,
    salary INTEGER
);

CREATE TABLE Inventory (
    productID INTEGER,
    location TEXT,
    quantity INTEGER
);"""

In [58]:
import os

input = ddl
prompt = 'Make SQLite query based on DDL and instruction.'
instruction = "What are the types of our products?"
text = (
    prompt
    + '### Instruction:\n'
    + instruction
    + '### Input:\n'
    + input
    + '### Response:\n'
)

In [59]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [60]:
output = model.generate(input_ids, max_length=1000, temperature=0.3, num_return_sequences=1)

In [61]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

match = re.search(r'### Response:\s*(.*)', generated_text)
response = match.group(1)
response

'SELECT category FROM Products;'

In [62]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect('/home/ksaff/Desktop/ttyd/ComplexDB.db')
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [63]:
result = execute_query(response)
print(f'User instruction: {instruction}')
print(f'model output: {response}')
'Result query:', result['data']

User instruction: What are the types of our products?
model output: SELECT category FROM Products;


('Result query:',
 [('Clothing',),
  ('Home',),
  ('Electronics',),
  ('Home',),
  ('Clothing',),
  ('Clothing',),
  ('Toys',),
  ('Clothing',),
  ('Electronics',),
  ('Books',),
  ('Home',),
  ('Electronics',),
  ('Toys',),
  ('Toys',),
  ('Books',),
  ('Electronics',),
  ('Books',),
  ('Clothing',),
  ('Home',),
  ('Electronics',),
  ('Clothing',),
  ('Books',),
  ('Toys',),
  ('Home',),
  ('Toys',),
  ('Clothing',),
  ('Electronics',),
  ('Books',),
  ('Home',),
  ('Books',),
  ('Books',),
  ('Electronics',),
  ('Home',),
  ('Toys',),
  ('Home',),
  ('Books',),
  ('Electronics',),
  ('Toys',),
  ('Books',),
  ('Toys',),
  ('Clothing',),
  ('Electronics',),
  ('Books',),
  ('Clothing',),
  ('Electronics',),
  ('Home',),
  ('Toys',),
  ('Books',),
  ('Books',),
  ('Clothing',),
  ('Electronics',),
  ('Toys',),
  ('Clothing',),
  ('Electronics',),
  ('Toys',),
  ('Electronics',),
  ('Home',),
  ('Home',),
  ('Clothing',),
  ('Electronics',),
  ('Clothing',),
  ('Toys',),
  ('Toys',),
 